## Import Dependencies and Data

In [3]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

In [4]:
df = pd.read_csv('diabetes.csv')

## Summarize Data

In [5]:
print('Dataframe head:\n', df.head(), '\n')
print('Dataframe tail:\n', df.tail(), '\n')
print('Data column summary statistics:\n', df.describe())

Dataframe head:
    Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1   

Dataframe tail:
      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72     

In [6]:
print('')

Data Cleaning and Preprocessing
Handle Missing Values: Check for missing or zero values (which seem to be present in 'SkinThickness', 'Insulin', etc.) that may need to be imputed or handled appropriately.
Data Transformation: Normalize or standardize the continuous features such as Glucose, BloodPressure, BMI, etc., as they seem to be on different scales.
Categorical Encoding: If there are any categorical variables not included in the sample provided, they should be encoded properly using techniques like one-hot encoding.

Exploratory Data Analysis (EDA)
Statistical Summary: Generate descriptive statistics to understand the distribution of the data.
Correlation Analysis: Check the correlation between features to identify multicollinearity.
Visualization: Create visual plots (like histograms, box plots, scatter plots) to see the distribution of features and the relationship between features and the outcome.

Model Selection

COMPARE TWO MODELS AND THEIR PERFORMANCE?

Model Training and Evaluation
Training: Train the model(s) on the training dataset.
Evaluation Metrics: Evaluate models using metrics such as accuracy, precision, recall, F1-score, ROC-AUC curve.

Validation
Validation Set: Evaluate the final model on a separate validation set that was not used during the model training phase to check for overfitting.
Learning Curves: Plot learning curves to diagnose if the model is learning well or if it's overfitting or underfitting.

Prediction and Interpretation
Make Predictions: Use the final model to make predictions on new data.
Interpret Results: Use tools like SHAP or LIME for model interpretation to understand how different features affect the outcome.

What features contributed most to the prediction?